<a href="https://colab.research.google.com/github/HamzaGbada/Numba-cuda/blob/main/Numba_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we will discuss how to do parallel computing on your GPU with CUDA using python!

The [NVIDIA® CUDA® Toolkit](https://developer.nvidia.com/cuda-toolkit) provides a development environment for creating high performance GPU-accelerated applications.

We will cover these sections:

1.   Cuda kernel
2.   Memory management
3.   Device function
4.   Threads and blocks configuration
5.   Application in image processing



Before we start, let's take a look on what is [Numba](https://numba.pydata.org/).

So Numba is a just-in-time compiler that translates python code into machine code using [LLVM](https://llvm.org/), which make numerical alogrithm in Python approach the speeds of C or FORTRAN.
 
